# PyTorch: Optimization & Training

https://github.com/pytorch/examples/tree/master/mnist

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1 # 데이터가 셔플해서 들어갈때 셔플을 하더라도 같이 섞는 것

batch_size = 64
test_batch_size = 64

no_cuda = False

In [3]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Preprocess

In [4]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

# Model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Optimization

- Model과 Optimization 설정

In [6]:
model = Net().to(device) # device에 담아준다

In [7]:
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.5)

- Parameters들 확인(굳이 보려면 이렇게 하면됨)
    - Weight, Bais를 순서대로 보여줌

In [8]:
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Before Training

- 학습하기 전에 Model이 Train할 수 있도록 Train Mode로 변환
    - Convolution 또는 Linear 뿐만 아니라, DropOut과 추후에 배우게 될 Batch Normalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비

In [9]:
model.train() # train mode

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch 데이터 추출

In [10]:
data, target = next(iter(train_loader))

In [11]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 Batch 데이터를 cpu 또는 gpu와 같은 device에 compile

In [12]:
data, target = data.to(device), target.to(device)

In [13]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비

In [14]:
optimizer.zero_grad()

- 준비한 데이터를 model에 input으로 넣어 output을 얻음

In [15]:
output = model(data)

- Model에서 예측한 결과를 Loss Function에 넣음
    - 여기 예제에서는 Negative Log-Likelihood Loss 라는 Loss Function을 사용

In [16]:
loss = F.nll_loss(output, target)

- Back Propagation을 통해 Gradients를 계산

In [17]:
loss.backward()

- 계산된 Gradients는 계산된 걸로 끝이 아니라 Parameter에 Update

In [18]:
optimizer.step()

# Start Training

위의 최적화 과정을 반복하여 학습 시작

In [19]:
epochs = 1
log_interval = 100 # 100개의 step이 거칠때마다 보여줄 것이다

In [21]:
for epoch in range(1, epochs+1):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 한번들어올때마다 인덱스가 들어온다
        data, target = data.to(device), target.to(device) # 데이터를 뽑았다
        optimizer.zero_grad() # gradient(경사도)를 clear
        output = model(data) # 앞에서 받은 데이터를 모델에 넣는다
        loss = F.nll_loss(output, target) # 비교해서 loss를 뽑음
        loss.backward() # 기울기 계산
        optimizer.step() # 업데이트
        
        if batch_idx % log_interval == 0: # 학습이 되는지 안되는지 로그를 보기
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                # 몇번째 데이터 인지 , 전체 데이터 수 반환 (len(data)는 배치사이즈)
                100 * batch_idx / len(train_loader), loss.item()
            ))

Train Epoch : 1 [0/60000 (0%)]	Loss : 2.317356
Train Epoch : 1 [6400/60000 (11%)]	Loss : 2.247029
Train Epoch : 1 [12800/60000 (21%)]	Loss : 2.150118
Train Epoch : 1 [19200/60000 (32%)]	Loss : 1.990300
Train Epoch : 1 [25600/60000 (43%)]	Loss : 1.534851
Train Epoch : 1 [32000/60000 (53%)]	Loss : 1.280391
Train Epoch : 1 [38400/60000 (64%)]	Loss : 0.878190
Train Epoch : 1 [44800/60000 (75%)]	Loss : 0.691761
Train Epoch : 1 [51200/60000 (85%)]	Loss : 0.747761
Train Epoch : 1 [57600/60000 (96%)]	Loss : 0.545452


# Evaluation

- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
    - Batch Normalization이나 Drop Out 같은 Layer들을 잠금

In [22]:
model.eval() # 평가모드 시작

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임

In [23]:
test_loss = 0
correct = 0

with torch.no_grad(): # 테스트 모드때는 꺼주는게 중요
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss += F.nll_loss(output, target, reduction='sum').item() # 로그를 보기 위해
    # reduction은 한번에 하나의 스칼라로 리턴해주며 계산해준다
    
    pred = output.argmax(dim=1, keepdim=True) # 컴퓨터가 생각하는 정답을 꺼내보고
    # 차원 수는 계속 유지
    correct = pred.eq(target.view_as(pred)).sum() # 같은 것끼리 True는 True끼리 더한다
    # 그것을 correct에 담는다
    

In [24]:
test_loss

30.502098083496094

In [27]:
output.shape

torch.Size([64, 10])

In [25]:
pred.shape

torch.Size([64, 1])

In [28]:
target.view_as(pred).shape

torch.Size([64, 1])

In [29]:
pred.eq(target.view_as(pred))

tensor([[ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False

In [30]:
pred.eq(target.view_as(pred)).sum()

tensor(56)

In [31]:
pred.eq(target.view_as(pred)).sum().item() / 64

0.875

In [32]:
test_loss /= len(test_loader.dataset)

In [33]:
test_loss

0.0030502098083496094

In [35]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))


Test set: Average Loss: 0.4849, Accuracy: 8650/10000 (86%)



## 완성본

In [36]:
for epoch in range(1, epochs+1):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 한번들어올때마다 인덱스가 들어온다
        data, target = data.to(device), target.to(device) # 데이터를 뽑았다
        optimizer.zero_grad() # gradient(경사도)를 clear
        output = model(data) # 앞에서 받은 데이터를 모델에 넣는다
        loss = F.nll_loss(output, target) # 비교해서 loss를 뽑음
        loss.backward() # 기울기 계산
        optimizer.step() # 업데이트
        
        if batch_idx % log_interval == 0: # 학습이 되는지 안되는지 로그를 보기
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                # 몇번째 데이터 인지 , 전체 데이터 수 반환 (len(data)는 배치사이즈)
                100 * batch_idx / len(train_loader), loss.item()
            ))
    # Test Mode        
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

Train Epoch : 1 [0/60000 (0%)]	Loss : 0.429992
Train Epoch : 1 [6400/60000 (11%)]	Loss : 0.646492
Train Epoch : 1 [12800/60000 (21%)]	Loss : 0.357494
Train Epoch : 1 [19200/60000 (32%)]	Loss : 0.395989
Train Epoch : 1 [25600/60000 (43%)]	Loss : 0.273421
Train Epoch : 1 [32000/60000 (53%)]	Loss : 0.269179
Train Epoch : 1 [38400/60000 (64%)]	Loss : 0.399411
Train Epoch : 1 [44800/60000 (75%)]	Loss : 0.503434
Train Epoch : 1 [51200/60000 (85%)]	Loss : 0.285299
Train Epoch : 1 [57600/60000 (96%)]	Loss : 0.245053

Test set: Average Loss: 0.2946, Accuracy: 9141/10000 (91%)

